In [3]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random

In [4]:
class Environment:
    def __init__(self):
        self.width_range = (0, 75)
        self.height_range = (0, 75)
        self.obstacles = self.generate_obstacles()

    @staticmethod
    def generate_obstacles(num_obstacles=4, min_size=5, max_size=20, max_position=100):
        obstacles = []

        for _ in range(num_obstacles):
            x = random.randint(0, max_position)
            y = random.randint(0, max_position)
            width = random.randint(min_size, max_size)
            height = random.randint(min_size, max_size)

            obstacle = [x, y, width, height]
            obstacles.append(obstacle)

        return obstacles


env = Environment()
print("Obstacles:", env.obstacles)

Obstacles: [[37, 89, 19, 7], [12, 79, 9, 20], [98, 29, 11, 12], [31, 13, 17, 15]]


In [9]:
class Node:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.parent = None
        self.cost = 0.0

class RRTStar:
    def __init__(self, environment, start, goal, max_iterations=2000, step_size=2.0, goal_radius=1.0, search_radius=5.0):
        self.env = environment
        self.start = Node(start[0], start[1])
        self.goal = Node(goal[0], goal[1])
        self.max_iterations = max_iterations
        self.step_size = step_size
        self.goal_radius = goal_radius
        self.search_radius = search_radius
        self.nodes = [self.start]
        self.path = []

    #FINDING DISTANCE BETWEEN TWO NODES
    def distance(self, node1, node2):
        return math.sqrt((node1.x - node2.x)**2 + (node1.y - node2.y)**2)
    
    #CHECKING COLLISION WITH OBSTACLES
    def is_collision_free(self, new_node, nearest_node):
        line = [[nearest_node.x, nearest_node.y], [new_node.x, new_node.y]]
        for obstacle in self.env.obstacles:
            if self.is_intersect(line, obstacle):
                return False

        for obstacle in self.env.obstacles:
            x, y, w, h = obstacle
            if (
                x <= new_node.x <= x + w and
                y <= new_node.y <= y + h
            ):
                return False

        return True

    def is_intersect(self, line, obstacle):
        x1, y1 = line[0]
        x2, y2 = line[1]
        x3, y3, w, h = obstacle
        x4, y4 = x3 + w, y3 + h

        if (x2 <= x3 or x1 >= x4 or y2 <= y3 or y1 >= y4):
            return False

        if (x1 >= x3 and x1 <= x4 and y1 >= y3 and y1 <= y4):
            return True

        if (x2 >= x3 and x2 <= x4 and y2 >= y3 and y2 <= y4):
            return True

        return ((x1 - x3) * (y4 - y3) - (y1 - y3) * (x4 - x3)) * ((x2 - x3) * (y4 - y3) - (y2 - y3) * (x4 - x3)) < 0

    # FINDING NEAREST NODE TO A RANDOM NODE
    def find_nearest_node(self, random_node):
        min_dist = float('inf')
        nearest_node = None

        for node in self.nodes:
            d = self.distance(node, random_node)
            if d < min_dist:
                min_dist = d
                nearest_node = node

        return nearest_node
    
    # FIND NODES WITHIN THE SEARCH RADIUS
    def find_near_nodes(self, random_node):
        near_nodes = [node for node in self.nodes if self.distance(node, random_node) < self.search_radius]
        return near_nodes

    def reform(self, new_node, near_nodes):
        for node in near_nodes:
            if node == new_node.parent:
                continue

            tentative_cost = node.cost + self.distance(node, new_node)

            if tentative_cost < new_node.cost and self.is_collision_free(new_node, node):
                new_node.parent = node
                new_node.cost = tentative_cost
                self.add_cost(new_node)
    
    # UPDATE COST
    def add_cost(self, parent_node):
        for node in self.nodes:
            if node.parent == parent_node:
                node.cost = parent_node.cost + self.distance(node, parent_node)
                self.add_cost(node)

    def generate_random_node(self):
        if random.random() < 0.05:  # 5% chance to select the goal as the random node
            return Node(self.goal.x, self.goal.y)
        else:
            x = random.uniform(self.env.width_range[0], self.env.width_range[1])
            y = random.uniform(self.env.height_range[0], self.env.height_range[1])
            return Node(x, y)

    def plan(self):
        # create random node and find its nearest node
        for i in range(self.max_iterations):
            random_node = self.generate_random_node()
            nearest_node = self.find_nearest_node(random_node)
            
            #check if distance between nearest node and the generated node is greater than step size
            if self.distance(nearest_node, random_node) > self.step_size:
                theta = math.atan2(random_node.y - nearest_node.y, random_node.x - nearest_node.x)
                new_node = Node(nearest_node.x + self.step_size * math.cos(theta),
                                nearest_node.y + self.step_size * math.sin(theta))
            else:
                new_node = random_node

            if not self.is_collision_free(new_node, nearest_node):
                continue
            
            #find nodes within the search radius
            near_nodes = self.find_near_nodes(new_node)
            new_node.parent = nearest_node
            new_node.cost = nearest_node.cost + self.distance(new_node, nearest_node)

            self.reform(new_node, near_nodes)
            self.nodes.append(new_node)
            
            # check if we reached near the goal
            if self.distance(new_node, self.goal) < self.goal_radius:
                self.goal.parent = new_node
                self.goal.cost = new_node.cost + self.distance(self.goal, new_node)
                self.add_cost(self.goal)
                break
    
    #finding the optimal path
    def extract_path(self):
        current_node = self.goal
        while current_node is not None:
            self.path.append((current_node.x, current_node.y))
            current_node = current_node.parent
        self.path.reverse()

    def plot(self):
        plt.figure(figsize=(10, 10))

        #plotting obstacles
        for obstacle in self.env.obstacles:
            obstacle_rect = plt.Rectangle((obstacle[0], obstacle[1]), obstacle[2], obstacle[3], fc='r')
            plt.gca().add_patch(obstacle_rect)

        #plotting all nodes generated
        for node in self.nodes:
            if node.parent:
                plt.plot([node.x, node.parent.x], [node.y, node.parent.y], '-o', color='#000000',)
        
        #plotting optimal path in green
        for i in range(len(self.path) - 1):
            plt.plot([self.path[i][0], self.path[i + 1][0]], [self.path[i][1], self.path[i + 1][1]], '-o', color='#00FF00', markersize=5)

        #plotting start and goal points
        plt.plot(self.start.x, self.start.y, 'mo', markersize=15, label='Start')
        plt.plot(self.goal.x, self.goal.y, 'co', markersize=15, label='Goal')

        plt.legend()
        plt.grid(True)
        plt.savefig('path_random_obstacles.jpg')
        plt.show()

In [ ]:
start_point = (1, 1)
goal_point = (70, 70)

env = Environment()
rrt_star = RRTStar(env, start_point, goal_point)

rrt_star.plan()

rrt_star.extract_path()
print("Optimal Path:", rrt_star.path)

rrt_star.plot()


Optimal Path: [(1, 1), (2.89673760160551, 4.520451738346925), (4.684817138952477, 7.962393410518921), (5.788407028649042, 11.32209380142015), (7.25569307995318, 14.968518880313912), (8.863039707926319, 16.158660391876037), (12.179599612279818, 19.241140268976217), (14.132580938033293, 19.672259673849393), (17.390794276755848, 21.929859391651195), (19.689249538465308, 25.10222220079388), (21.172518487255733, 27.642235978027117), (23.889160777169575, 30.003666681916997), (27.274687530197017, 31.32112803177477), (29.875624643659265, 34.32154898012727), (31.51553448821751, 37.64524697763388), (32.92718399820648, 39.06201995201895), (34.78516638601736, 42.36787304005034), (36.35859872296816, 43.602502814937345), (38.6399231754973, 47.214153691443265), (42.16213454531072, 48.92016543287472), (45.565783361921895, 50.965427388870175), (47.14354286426263, 52.194522565320526), (50.51966200409744, 52.82387530210948), (51.76122842944549, 54.39183984617559), (54.076708270829116, 58.05426733917894),